**IBM QISKIT CODE**

In [10]:
!pip install qiskit qiskit-aer pylatexenc ipywidgets

A **basic variational circuit** is a **parameterized quantum circuit** — meaning its gates depend on tunable parameters (usually **angles**). These parameters are optimized using classical algorithms during **Variational Quantum Algorithms (VQAs)** like:

* Variational Quantum Eigensolver (VQE)
* Quantum Approximate Optimization Algorithm (QAOA)

Why rotation gates?

Rotation gates like:

* `Rx(θ)` – rotation around the X-axis
* `Ry(θ)` – rotation around the Y-axis
* `Rz(θ)` – rotation around the Z-axis

are **continuously tunable** via the angle **θ**, and hence they are:

* **Parameterized** – controlled by real numbers
* **Flexible** – can explore any point on the Bloch sphere
* **Trainable** – perfect for variational circuits where classical optimization algorithms adjust angles to minimize a cost function

Why not use H, X, Y, or Z gates?

These are **fixed operations** (non-parameterized):

* Hadamard (H) is a fixed superposition operation.
* X, Y, Z are fixed π rotations.

They **can't be tuned**, so they **aren’t suitable for training** in variational algorithms — though sometimes they're used to **initialize** states or add fixed behavior.

A **basic variational circuit** typically consists of **rotation gates (Rx, Ry, Rz)** and **entangling gates (like CNOT)**. The rotation angles (parameters) are **tunable** and optimized during the learning process. That’s why they are essential.

In [17]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_vector, circuit_drawer
from qiskit.quantum_info import Statevector, partial_trace
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

angle_options = [
    ('-2π', -2*np.pi),
    ('-3π/2', -3*np.pi/2),
    ('-π', -np.pi),
    ('-π/2', -np.pi/2),
    ('-π/4', -np.pi/4),
    ('0', 0),
    ('π/4', np.pi/4),
    ('π/2', np.pi/2),
    ('π', np.pi),
    ('3π/2', 3*np.pi/2),
    ('2π', 2*np.pi),
]

theta_dropdown = widgets.Dropdown(options=angle_options, description='θ (Rx):', value=0)
phi_dropdown = widgets.Dropdown(options=angle_options, description='φ (Ry):', value=0)
lam_dropdown = widgets.Dropdown(options=angle_options, description='λ (Rz):', value=0)

output = widgets.Output()

def run_circuit(theta, phi, lam):
    qc = QuantumCircuit(1, 1)
    qc.rx(theta, 0)
    qc.ry(phi, 0)
    qc.rz(lam, 0)
    qc.measure(0, 0)

    sim = AerSimulator()
    compiled = transpile(qc, sim)
    result = sim.run(compiled, shots=1024).result()
    counts = result.get_counts()

    return qc, counts

def update(change=None):
    with output:
        clear_output(wait=True)
        theta = theta_dropdown.value
        phi = phi_dropdown.value
        lam = lam_dropdown.value

        qc, counts = run_circuit(theta, phi, lam)
        display(qc.draw('mpl'))
        display(plot_histogram(counts, title="Measurement Result"))

theta_dropdown.observe(update, names='value')
phi_dropdown.observe(update, names='value')
lam_dropdown.observe(update, names='value')

display(widgets.HBox([theta_dropdown, phi_dropdown, lam_dropdown]))
display(output)

update()


Output()

In [21]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_vector
from qiskit.quantum_info import Statevector, partial_trace
from IPython.display import display, clear_output
import ipywidgets as widgets

angle_options = [
    ('-2π', -2*np.pi),
    ('-3π/2', -3*np.pi/2),
    ('-π', -np.pi),
    ('-π/2', -np.pi/2),
    ('-π/4', -np.pi/4),
    ('0', 0),
    ('π/4', np.pi/4),
    ('π/2', np.pi/2),
    ('π', np.pi),
    ('3π/2', 3*np.pi/2),
    ('2π', 2*np.pi),
]

theta0_dd = widgets.Dropdown(options=angle_options, description='Q0 θ (Rx):', value=0)
phi0_dd = widgets.Dropdown(options=angle_options, description='Q0 φ (Ry):', value=0)
lam0_dd = widgets.Dropdown(options=angle_options, description='Q0 λ (Rz):', value=0)

theta1_dd = widgets.Dropdown(options=angle_options, description='Q1 θ (Rx):', value=0)
phi1_dd = widgets.Dropdown(options=angle_options, description='Q1 φ (Ry):', value=0)
lam1_dd = widgets.Dropdown(options=angle_options, description='Q1 λ (Rz):', value=0)

output = widgets.Output()

def get_bloch_vector(rho):
    """Calculate Bloch vector from single-qubit density matrix"""
    x = 2 * np.real(rho[0, 1])
    y = 2 * np.imag(rho[1, 0])
    z = np.real(rho[0, 0] - rho[1, 1])
    return [x, y, z]

def run_variational_entangle_circuit(t0, p0, l0, t1, p1, l1):
    qc = QuantumCircuit(2, 2)

    qc.rx(t0, 0)
    qc.ry(p0, 0)
    qc.rz(l0, 0)

    qc.rx(t1, 1)
    qc.ry(p1, 1)
    qc.rz(l1, 1)

    sv_before = Statevector.from_instruction(qc)
    rho0_before = partial_trace(sv_before, [1]).data
    rho1_before = partial_trace(sv_before, [0]).data
    bloch0_before = get_bloch_vector(rho0_before)
    bloch1_before = get_bloch_vector(rho1_before)

    qc.cx(0, 1)

    qc.measure([0, 1], [0, 1])

    sv_after = Statevector.from_instruction(qc.remove_final_measurements(inplace=False))
    rho0_after = partial_trace(sv_after, [1]).data
    rho1_after = partial_trace(sv_after, [0]).data
    bloch0_after = get_bloch_vector(rho0_after)
    bloch1_after = get_bloch_vector(rho1_after)

    sim = AerSimulator()
    compiled = transpile(qc, sim)
    result = sim.run(compiled, shots=1024).result()
    counts = result.get_counts()

    return qc, counts, bloch0_before, bloch1_before, bloch0_after, bloch1_after

def update(change=None):
    with output:
        clear_output(wait=True)

        qc, counts, b0b, b1b, b0a, b1a = run_variational_entangle_circuit(
            theta0_dd.value, phi0_dd.value, lam0_dd.value,
            theta1_dd.value, phi1_dd.value, lam1_dd.value
        )

        display(qc.draw('mpl'))
        display(plot_histogram(counts, title="Measurement Result"))

        display(plot_bloch_vector(b0b, title="Qubit 0 Bloch Sphere BEFORE Entanglement"))
        display(plot_bloch_vector(b1b, title="Qubit 1 Bloch Sphere BEFORE Entanglement"))

        display(plot_bloch_vector(b0a, title="Qubit 0 Bloch Sphere AFTER Entanglement"))
        display(plot_bloch_vector(b1a, title="Qubit 1 Bloch Sphere AFTER Entanglement"))

for dd in [theta0_dd, phi0_dd, lam0_dd, theta1_dd, phi1_dd, lam1_dd]:
    dd.observe(update, names='value')

display(widgets.VBox([
    widgets.HBox([theta0_dd, phi0_dd, lam0_dd]),
    widgets.HBox([theta1_dd, phi1_dd, lam1_dd])
]))
display(output)

update()


Output()

**PENNYLANE CODE**

In [13]:
!pip install pennylane

In [19]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

angle_options = [
    ('-2π', -2*np.pi),
    ('-3π/2', -3*np.pi/2),
    ('-π', -np.pi),
    ('-π/2', -np.pi/2),
    ('-π/4', -np.pi/4),
    ('0', 0),
    ('π/4', np.pi/4),
    ('π/2', np.pi/2),
    ('π', np.pi),
    ('3π/2', 3*np.pi/2),
    ('2π', 2*np.pi),
]

theta0_dd = widgets.Dropdown(options=angle_options, description='Q0 Rx:', value=0)
phi0_dd   = widgets.Dropdown(options=angle_options, description='Q0 Ry:', value=0)
lam0_dd   = widgets.Dropdown(options=angle_options, description='Q0 Rz:', value=0)

theta1_dd = widgets.Dropdown(options=angle_options, description='Q1 Rx:', value=0)
phi1_dd   = widgets.Dropdown(options=angle_options, description='Q1 Ry:', value=0)
lam1_dd   = widgets.Dropdown(options=angle_options, description='Q1 Rz:', value=0)

output = widgets.Output()

dev = qml.device("default.qubit", wires=2, shots=1024)

@qml.qnode(dev)
def circuit(t0, p0, l0, t1, p1, l1):
    qml.RX(t0, wires=0)
    qml.RY(p0, wires=0)
    qml.RZ(l0, wires=0)

    qml.RX(t1, wires=1)
    qml.RY(p1, wires=1)
    qml.RZ(l1, wires=1)

    qml.CNOT(wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

def draw_circuit():
    @qml.qnode(dev)
    def dummy_circuit():
        qml.RX(theta0_dd.value, wires=0)
        qml.RY(phi0_dd.value, wires=0)
        qml.RZ(lam0_dd.value, wires=0)

        qml.RX(theta1_dd.value, wires=1)
        qml.RY(phi1_dd.value, wires=1)
        qml.RZ(lam1_dd.value, wires=1)

        qml.CNOT(wires=[0, 1])
        return qml.state()

    drawn = qml.draw(dummy_circuit)
    print(drawn())

def update(change=None):
    with output:
        clear_output(wait=True)

        draw_circuit()

        samples = circuit(theta0_dd.value, phi0_dd.value, lam0_dd.value,
                          theta1_dd.value, phi1_dd.value, lam1_dd.value)

        bitstrings = [(0 if z0 > 0 else 1, 0 if z1 > 0 else 1) for z0, z1 in zip(*samples)]
        from collections import Counter
        counts = Counter([f"{a}{b}" for a, b in bitstrings])

        plt.figure(figsize=(6,4))
        plt.bar(counts.keys(), counts.values())
        plt.title("Measurement Result")
        plt.xlabel("Bitstring")
        plt.ylabel("Counts")
        plt.show()

for dd in [theta0_dd, phi0_dd, lam0_dd, theta1_dd, phi1_dd, lam1_dd]:
    dd.observe(update, names='value')

display(widgets.VBox([
    widgets.HBox([theta0_dd, phi0_dd, lam0_dd]),
    widgets.HBox([theta1_dd, phi1_dd, lam1_dd])
]))
display(output)
update()


/usr/local/lib/python3.12/dist-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


Output()

**GOOGLE CIRQ CODE**

In [15]:
!pip install cirq

In [20]:
import cirq
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import Counter

angle_options = [
    ('-2π', -2*np.pi),
    ('-3π/2', -3*np.pi/2),
    ('-π', -np.pi),
    ('-π/2', -np.pi/2),
    ('-π/4', -np.pi/4),
    ('0', 0),
    ('π/4', np.pi/4),
    ('π/2', np.pi/2),
    ('π', np.pi),
    ('3π/2', 3*np.pi/2),
    ('2π', 2*np.pi),
]

theta0_dd = widgets.Dropdown(options=angle_options, description='Q0 Rx:', value=0)
phi0_dd = widgets.Dropdown(options=angle_options, description='Q0 Ry:', value=0)
lam0_dd = widgets.Dropdown(options=angle_options, description='Q0 Rz:', value=0)

theta1_dd = widgets.Dropdown(options=angle_options, description='Q1 Rx:', value=0)
phi1_dd = widgets.Dropdown(options=angle_options, description='Q1 Ry:', value=0)
lam1_dd = widgets.Dropdown(options=angle_options, description='Q1 Rz:', value=0)

output = widgets.Output()

q0, q1 = cirq.LineQubit.range(2)

def build_circuit(t0, p0, l0, t1, p1, l1):
    circuit = cirq.Circuit()
    circuit.append(cirq.rx(t0)(q0))
    circuit.append(cirq.ry(p0)(q0))
    circuit.append(cirq.rz(l0)(q0))

    circuit.append(cirq.rx(t1)(q1))
    circuit.append(cirq.ry(p1)(q1))
    circuit.append(cirq.rz(l1)(q1))

    circuit.append(cirq.CNOT(q0, q1))
    circuit.append(cirq.measure(q0, key='q0'))
    circuit.append(cirq.measure(q1, key='q1'))
    return circuit

def update(change=None):
    with output:
        clear_output(wait=True)

        circuit = build_circuit(theta0_dd.value, phi0_dd.value, lam0_dd.value,
                                theta1_dd.value, phi1_dd.value, lam1_dd.value)
        print(circuit)

        sim = cirq.Simulator()
        result = sim.run(circuit, repetitions=1024)

        q0_bits = result.measurements['q0'][:, 0]
        q1_bits = result.measurements['q1'][:, 0]
        bitstrings = [f"{q0}{q1}" for q0, q1 in zip(q0_bits, q1_bits)]
        counts = Counter(bitstrings)

        plt.figure(figsize=(6, 4))
        plt.bar(counts.keys(), counts.values())
        plt.title("Measurement Result")
        plt.xlabel("Bitstring")
        plt.ylabel("Counts")
        plt.show()

for dd in [theta0_dd, phi0_dd, lam0_dd, theta1_dd, phi1_dd, lam1_dd]:
    dd.observe(update, names='value')

display(widgets.VBox([
    widgets.HBox([theta0_dd, phi0_dd, lam0_dd]),
    widgets.HBox([theta1_dd, phi1_dd, lam1_dd])
]))
display(output)
update()


Output()